In [1]:
#Import functions
import polars as pl
from functools import partial
from itertools import product, combinations
import multiprocessing
from numba import jit
import numpy as np
import pandas as pd

In [2]:
#Load prices and timestamps from CSVs, just AAPL for now, TSLA soon
currentDf=pl.read_csv("../Data/SP500/minuteHist2021/tradingHours/AAPL.csv", infer_schema_length=None)
openPrices=currentDf['open'].to_numpy()
closePrices=currentDf['close'].to_numpy()
timestamps=currentDf['time'].to_numpy()

In [3]:
#Join data into one 2d array
dataArray = np.dstack((openPrices,closePrices, timestamps))[0]

In [4]:
#Average change between ticks
totalChange=0
changeList=[]
totalK=0
for k in range(len(dataArray)-1):
    if dataArray[k+1][2]-dataArray[k][2]==60000:
        tempChange = abs((openPrices[k+1]/openPrices[k])-1)
        totalChange += tempChange
        changeList.append(tempChange)
        totalK+=1
        if tempChange>0.02:
            print(k)
changeSeries = pd.Series(changeList)
averageChange=totalChange/totalK
sortedDiffs = sorted(changeList.copy())

129251
131620
134445
182198


In [31]:
#Statistics of data
print(changeSeries.describe())
print(averageChange)

count    483348.000000
mean          0.000523
std           0.000619
min           0.000000
25%           0.000152
50%           0.000344
75%           0.000674
max           0.028231
dtype: float64
0.0005232034661379928


In [9]:
#Algo
#Numba for speed
@jit(nopython=True)
#Takes in list of paramaters and an array with all the data
def algo(paramsList, dataArray):
    fee=paramsList[0][0] #Percent change in price to look for
    timeout=int(paramsList[0][1]) #The timeout when it takes too long
    divisor=int(paramsList[0][2]) #Divisor for ema
    allParams=paramsList[1:]
    
    #params has an array of arrays with ticks before, change, version
    
    tradePercent=0
    totalReturn=1

    total=0

    start=60 #Start of data
    end=int(len(openPrices)-61) #End of data

    for j in range(start, end):

        #Check that its not within the first or last hour of trading
        if dataArray[j][2]-dataArray[j-60][2]<4500000 and dataArray[j+60][2]-dataArray[j][2]<4500000:

            #Set start price and continue
            startPrice=dataArray[j][0]
            continueParams=True

            for param in allParams:
                ticksBefore = int(param[0])
                change = param[1]
                version = param[2]

                #Set price we are checking
                if version==1:
                    checkPrice=dataArray[j-ticksBefore][0]


                elif version==2:

                    checkTotal = 0
                    #Exponential average by using equation of (1/2 * 1/2^n * price) (a/1-r) (1/3 * 2/3^n * price) which = 1 as it tends to infinity
                    expoError=0
                    divisor=4

                    count=0
                    for k in range(j-ticksBefore+1,j+1):
                        checkTotal+=(int(dataArray[k][0]) * (1/divisor * (1-(1/divisor))**(ticksBefore-count)))
                        count+=1

                    count=0
                    for k in range(j-ticksBefore+1,j+1):
                        expoError+=(1/divisor * (1-(1/divisor))**(ticksBefore-count))
                        count+=1

                    checkPrice = checkTotal/expoError


                elif version==3:
                    #checkPrice=sma(ticksBefore, j, openPrices)
                    checkTotal = 0
                    #Simple mean by add and divide
                    for k in range(j-ticksBefore+1,j+1):
                        checkTotal+=int(dataArray[k][0])
                    checkPrice = checkTotal/ticksBefore


                #If price now is above price we are checking, then continue
                if not checkPrice*(change+0.00002)>startPrice>checkPrice*change:
                    continueParams = False

            if continueParams:

                #Add to tally
                total+=1

                #Return on investment = new price / old price

                returnValue = dataArray[j+timeout][0]/dataArray[j][0]
                totalReturn*=(returnValue-fee)
                tradePercent+=(returnValue-fee)
        
    if total>0:
        tradePercent=tradePercent/total
        
    #Return the average trade percent, the total number of times, and the parameters
    returnList = [tradePercent, totalReturn, total] + [param for param in allParams for param in param]
    return returnList
    # print(tradePercent)
    # print(tradePerDay)

#algo with all the dataframes already passed through
partialAlgo = partial(algo, dataArray=dataArray)

In [10]:
#Test algo to compile it
algo(np.array([[0, 50.00, 0],[10, 1.0, 3],[20, 1.00, 2]]), dataArray)

[1.0002004781001128, 1.0053308478795735, 29.0, 10.0, 1.0, 3.0, 20.0, 1.0, 2.0]

In [25]:
#Params
#rough commission fee
fee=[0.00]

#change
changeList=np.arange(1.000,1.001,0.00002)

#timeout
timeoutList=[1,2,3,5,10,20,30,50]

#ema sma price before
maTicksBeforeList=[3,5,10,20,50]
#days before list
priceTicksBeforeList=[1,2,3,5,10,20,30,50]


priceParamProduct = list(product(priceTicksBeforeList, changeList, [1]))
maParamProduct = list(product(maTicksBeforeList, changeList, [2,3]))
#maParamProduct = list(product(maTicksBeforeList, changeList, [3]))
allParamProduct = priceParamProduct + maParamProduct

allSettingsProduct = list(product(fee, timeoutList, fee))

depth = 2
xParamCombinations = list(combinations(allParamProduct, depth))

fullCombinations=[]
for a in allSettingsProduct:
    for b in xParamCombinations:
        fullCombinations.append(np.array(([a] + [c for c in b])))
len(fullCombinations)

3236400

In [26]:
#Run the algo over the full combinations list with multiprocessing

#List of results to store in right order
allResults=[]

#Start multiprocessing
with multiprocessing.Pool(40) as pool:
    for result in pool.map(
        partialAlgo,
        fullCombinations
    ):
        #Check if the total tally is high enough to be reasonable
        allResults.append(result)

In [32]:
minTotal=10
results=[]

for result in allResults:
    if result[2]>=minTotal:
        results.append(result)

In [33]:
#Sort results by each trade and by total
usedResults = results

sortedResults = sorted(usedResults.copy(), reverse=True)
sortedTotalResults = sorted(usedResults.copy(), reverse=True, key=lambda x: x[1])

In [34]:
#Return, Total return, number of times, ticks before, change, version: 1-price 2-ema 3-sma
for k in range(20):
    print(sortedResults[k])

[1.004685145131974, 1.0469137987017665, 10.0, 5.0, 1.000219999999999, 1.0, 50.0, 1.000439999999998, 1.0]
[1.0044912637720123, 1.0549640102000963, 12.0, 1.0, 1.0005399999999975, 1.0, 5.0, 1.000439999999998, 1.0]
[1.0044802313388412, 1.0502756345419548, 11.0, 1.0, 1.0001599999999993, 1.0, 50.0, 1.000859999999996, 3.0]
[1.0041832306468743, 1.0552657318325063, 13.0, 3.0, 1.0000999999999995, 1.0, 30.0, 1.0001599999999993, 1.0]
[1.0041723299807586, 1.0597036800248283, 14.0, 2.0, 1.000899999999996, 1.0, 5.0, 1.0005799999999974, 1.0]
[1.0040210508999388, 1.0408376096901741, 10.0, 1.0, 1.0004799999999978, 1.0, 10.0, 1.0001199999999995, 1.0]
[1.0039488350147265, 1.0520907440040566, 13.0, 5.0, 1.0005199999999976, 1.0, 20.0, 1.00002, 1.0]
[1.0039016731607429, 1.03962267254821, 10.0, 1.0, 1.0007399999999966, 1.0, 5.0, 1.000239999999999, 1.0]
[1.003898769145042, 1.068251593199394, 17.0, 3.0, 1.000899999999996, 1.0, 5.0, 1.000859999999996, 1.0]
[1.003836336061171, 1.0390167289597387, 10.0, 1.0, 1.000

In [35]:
for k in range(20):
    print(sortedTotalResults[k])

[1.000288560977364, 1.2223496986289841, 715.0, 3.0, 1.0007799999999965, 2.0, 3.0, 1.0007799999999965, 3.0]
[1.0004279286635493, 1.2055365585111617, 444.0, 5.0, 1.000879999999996, 3.0, 10.0, 1.000879999999996, 3.0]
[1.000309857340655, 1.2003880952130983, 607.0, 3.0, 1.0009399999999957, 2.0, 5.0, 1.0009399999999957, 2.0]
[1.000309857340655, 1.2003880952130983, 607.0, 3.0, 1.0009399999999957, 2.0, 5.0, 1.0009399999999957, 3.0]
[1.000309857340655, 1.2003880952130983, 607.0, 3.0, 1.0009399999999957, 3.0, 5.0, 1.0009399999999957, 2.0]
[1.000309857340655, 1.2003880952130983, 607.0, 3.0, 1.0009399999999957, 3.0, 5.0, 1.0009399999999957, 3.0]
[1.000309857340655, 1.2003880952130983, 607.0, 5.0, 1.0009399999999957, 2.0, 5.0, 1.0009399999999957, 3.0]
[1.0002936471311907, 1.2003070988517484, 641.0, 3.0, 1.000899999999996, 2.0, 5.0, 1.000899999999996, 2.0]
[1.0002725024504149, 1.1936222457488113, 664.0, 3.0, 1.0007999999999964, 2.0, 5.0, 1.0007999999999964, 2.0]
[1.0002725024504149, 1.19362224574881